In [19]:
%pip install langchain-text-splitters transformers pandas numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [20]:
import numpy as np
import pandas as pd

In [21]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter

# Load a tokenizer for a BERT-like model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def count_tokens(text):
    return len(tokenizer.encode(text))

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=400,
    chunk_overlap=50,
    # length_function=len,
    length_function=count_tokens,
    # is_separator_regex=False,
)

In [22]:
def chunk(text):
    texts = text_splitter.create_documents([text])
    chunks = []
    for i,t in enumerate(texts):
        # replace newlines with spaces this can help keep word boundires
        chunks.append(t.page_content.replace("\n", " "))
    return chunks

In [34]:
df_p = pd.read_csv('./datasets/positive_w_text.csv', delimiter='|')
df_n = pd.read_csv('./datasets/negitive_w_text.csv', delimiter='|')

print(f'Positive: {len(df_p)}')
print(f'Negitive: {len(df_n)}')

df_p = df_p.drop(columns=["SUBJECT_ID"])
df_n = df_n.drop(columns=["SUBJECT_ID"])

df = pd.concat([df_p, df_n], ignore_index=True)
# df.reset_index(inplace=True)

# This shuffles the dataframe, but in a non-reproducable way
# df = df.sample(frac=1).reset_index(drop=True)
# Instead shuffle dataframe, but in a reproducable way
# SEED=42
# np.random.seed(SEED)
# idx = df.index.to_list()
# np.random.shuffle(idx)
# res = df.loc[idx].reset_index(drop=True)

df = df.sample(frac=1).reset_index(drop=True)
# display(df)

Positive: 4492
Negitive: 4000


In [24]:
from tables import classes, schema_dicts
print(classes)
print(schema_dicts)

['Intellectual Developmental Disorders', 'Communication Disorders', 'Autism Spectrum Disorder', 'Attention-Deficit/Hyperactivity Disorder', 'Specific Learning Disorder', 'Motor Disorders', 'Other Neurodevelo mental Disorders p', 'Schizophrenia Spectrum and Other Psychotic Disorders', 'Catatonia', 'Bipolar and Related Disorders', 'Depressive Disorders', 'Anxiety Disorders', 'Obsessive-Compulsive and Related Disorders', 'Trauma- and Stressor-Related Disorders', 'Dissociative Disorders', 'Somatic Symptom and Related Disorders', 'Feeding and Eating Disorders', 'Elimination Disorders', 'Sleep-Wake Disorders', 'Breathing-Related Sleep Disorders', 'Parasomnias', 'Sexual Dysfunctions', 'Gender Dysphoria', 'Disruptive, Impulse-Control, and Conduct Disorders', 'Neurocognitive Disorders', 'Personality Disorders', 'Cluster A Personality Disorders', 'Cluster B Personality Disorders', 'Cluster C Personalit Disorders y', 'Other Personality Disorders', 'Paraphilic Disorders', 'Other Mental Disorders a

In [25]:
schema_index = 0
max_avg_distance = 20

In [26]:
import weaviate
import weaviate.classes as wvc

def query_top_x(text, limit=1, max_distance=20):
    client = weaviate.connect_to_local()
    res = None
    try:
        collection = client.collections.use(schema_dicts[schema_index]["schema"])

        results = collection.query.near_text(
            # Because near_text is a purely vector search, we get a
            # distance and no score
            return_metadata=wvc.query.MetadataQuery(
                score=True, explain_score=True, distance=True, certainty=True
            ),
            query=text,
            limit=limit
        )
        for r in results.objects:
            if r.metadata.distance < max_distance:
                res = [classes.index(r.properties['title']), r.metadata.distance]

        return res
    finally:
        client.close()

In [27]:
# item = 1

# chart_chunks = chunk(df.iloc[item]["chart_text"])
# print(f"Chunks size: {len(chart_chunks)}")

# scores = []
# for c in chart_chunks:
#     d = query_top_five(c)
#     if d is not None:
#         scores.append(d)

# print(f"Scores size: {len(scores)}")

In [28]:
def score_document(s, max_dist=10):
    cat = {}
    top_results = []

    for i, _ in enumerate(classes):
        cat[i] = {"count":0, "distance":0}

    for _, d in enumerate(s):
        cat[d[0]]["count"] += 1
        cat[d[0]]["distance"] += d[1]
        cat[d[0]]["avg"] = cat[d[0]]["distance"] / cat[d[0]]["count"]

    for i, _ in enumerate(classes):
        if cat[i]["count"] > 0 and cat[i]["avg"] < max_dist:
            top_results.append([cat[i]["avg"], i])

    return top_results

# top = np.array(score_document(scores, max_avg_distance))
# print(np.sort(top, axis=0))
# print('--------------')
# print(f'{df.iloc[item]["class_number"]:02} {classes[df.iloc[item]["class_number"]]}')

In [29]:
test_range = 30

results = []
for i in range(test_range): # 4353//4):
    chart_chunks = chunk(df.iloc[i]["chart_text"])
    print(f"Chunks size: {len(chart_chunks)}")

    scores = []
    for c in chart_chunks:
        d = query_top_x(c, 5)
        if d is not None:
            scores.append(d)

    top = np.array(score_document(scores, max_avg_distance))
    top = np.sort(top, axis=0)

    results.append([i, df.iloc[i]["class_number"], top])

Chunks size: 85
Chunks size: 19
Chunks size: 852
Chunks size: 101
Chunks size: 190
Chunks size: 11
Chunks size: 44
Chunks size: 14
Chunks size: 32
Chunks size: 63


Created a chunk of size 472, which is longer than the specified 400


Chunks size: 267
Chunks size: 19
Chunks size: 108
Chunks size: 314
Chunks size: 62
Chunks size: 119
Chunks size: 28
Chunks size: 19


Created a chunk of size 439, which is longer than the specified 400
Created a chunk of size 439, which is longer than the specified 400
Created a chunk of size 439, which is longer than the specified 400
Created a chunk of size 439, which is longer than the specified 400
Created a chunk of size 439, which is longer than the specified 400
Created a chunk of size 439, which is longer than the specified 400
Created a chunk of size 439, which is longer than the specified 400
Created a chunk of size 439, which is longer than the specified 400


Chunks size: 1146
Chunks size: 14
Chunks size: 700
Chunks size: 32
Chunks size: 14
Chunks size: 19
Chunks size: 65
Chunks size: 12
Chunks size: 100
Chunks size: 39
Chunks size: 1133
Chunks size: 14


## Scoring

Correct answer in the top 5

In [33]:
df_results = pd.DataFrame(results, columns=["record", "original", "predict"])

def top5(x):
    return np.array([y[1] for i,y in enumerate(x) if i < 5])

def in_top_5(orig, top5):
    return 1 if orig in top5 else 0

def correct_predict(orig, top5):
    if orig == 0 and 11 not in top5 and 10 not in top5:
        return 1
    return 1 if orig in top5 else 0

df_results['top5'] = df_results['predict'].map(top5)
df_results['inTop5'] = df_results.apply(lambda x: in_top_5(orig=x['original'], top5=x['top5']), axis=1)
df_results['correct'] = df_results.apply(lambda x: correct_predict(orig=x['original'], top5=x['top5']), axis=1)

print(f"Correct top 5    {df_results['inTop5'].sum() / test_range}")
print(f"Correct category {df_results['correct'].sum() / test_range}")
display(df_results)

# Sample size 30
# i     %   MMR
# 0   .70  .20
# 1   .60  .16
# 2   .53  .14
# 3   .66  .18


Correct top 5    0.3
Correct category 0.43333333333333335


,record,original,predict,top5,inTop5,correct
0,0,10.0,"[[6.399860382080078, 5.0], [7.602376818656921,...","[5.0, 7.0, 8.0, 9.0, 10.0]",1,1
1,1,11.0,"[[7.625951766967773, 5.0], [7.702698707580566,...","[5.0, 10.0, 11.0, 12.0, 15.0]",1,1
2,2,0.0,"[[4.9907989501953125, 5.0], [7.476325988769531...","[5.0, 7.0, 9.0, 10.0, 11.0]",0,0
3,3,0.0,"[[5.557958398546491, 3.0], [6.583929538726807,...","[3.0, 9.0, 10.0, 11.0, 13.0]",0,0
4,4,0.0,"[[6.233088970184326, 9.0], [6.4432171474803575...","[9.0, 10.0, 11.0, 12.0, 13.0]",0,0
5,5,10.0,"[[6.9363861083984375, 7.0], [7.471152305603027...","[7.0, 9.0, 16.0, 19.0, 23.0]",0,0
6,6,0.0,"[[6.39496340070452, 9.0], [7.475881576538086, ...","[9.0, 13.0, 16.0, 18.0, 19.0]",0,1
7,7,10.0,"[[6.649440050125122, 9.0], [7.543604850769043,...","[9.0, 11.0, 12.0, 17.0, 19.0]",0,0
8,8,10.0,"[[6.190422058105469, 5.0], [6.820960998535156,...","[5.0, 9.0, 10.0, 11.0, 16.0]",1,1
9,9,10.0,"[[6.237871766090393, 5.0], [6.795262098312378,...","[5.0, 7.0, 9.0, 11.0, 15.0]",0,0


### Mean Reciprocal Rank (MRR)

The position of the first correct result is important

For each query, find the rank of the first correct result. The Reciprocal Rank (RR) is $1 / rank$. MRR is the average RR across all queries.

Example: Suppose you have 3 queries:
- Query 1: Correct result is ranked 1st → RR = 1/1 = 1.0
- Query 2: Correct result is ranked 3rd → RR = 1/3 ≈ 0.33
- Query 3: Correct result is ranked 2nd → RR = 1/2 = 0.5

$$
MRR = (1.0 + 0.33 + 0.5) / 3 ≈ 0.61
$$

$$
MRR = \frac{1}{∣Q∣} \sum_{i=1}^{∣Q∣}\frac{​1}{rank_i}
$$

A good Mean Reciprocal Rank (MRR) value typically ranges from 0.6 to 1.0

In [31]:
def mmr(orig, top5):
    idx = np.where(top5 == orig)[0]
    return (1/(idx+1) if idx.size != 0 else [0])[0]

df_results['mmr'] = df_results.apply(lambda x: mmr(orig=x['original'], top5=x['top5']), axis=1)

print(f"MMR: {df_results['mmr'].sum() / test_range}")
display(df_results)

MMR: 0.10055555555555555


,record,original,predict,top5,inTop5,correct,mmr
0,0,10.0,"[[6.399860382080078, 5.0], [7.602376818656921,...","[5.0, 7.0, 8.0, 9.0, 10.0]",1,1,0.200000
1,1,11.0,"[[7.625951766967773, 5.0], [7.702698707580566,...","[5.0, 10.0, 11.0, 12.0, 15.0]",1,1,0.333333
2,2,0.0,"[[4.9907989501953125, 5.0], [7.476325988769531...","[5.0, 7.0, 9.0, 10.0, 11.0]",0,0,0.000000
3,3,0.0,"[[5.557958398546491, 3.0], [6.583929538726807,...","[3.0, 9.0, 10.0, 11.0, 13.0]",0,0,0.000000
4,4,0.0,"[[6.233088970184326, 9.0], [6.4432171474803575...","[9.0, 10.0, 11.0, 12.0, 13.0]",0,0,0.000000
5,5,10.0,"[[6.9363861083984375, 7.0], [7.471152305603027...","[7.0, 9.0, 16.0, 19.0, 23.0]",0,0,0.000000
6,6,0.0,"[[6.39496340070452, 9.0], [7.475881576538086, ...","[9.0, 13.0, 16.0, 18.0, 19.0]",0,1,0.000000
7,7,10.0,"[[6.649440050125122, 9.0], [7.543604850769043,...","[9.0, 11.0, 12.0, 17.0, 19.0]",0,0,0.000000
8,8,10.0,"[[6.190422058105469, 5.0], [6.820960998535156,...","[5.0, 9.0, 10.0, 11.0, 16.0]",1,1,0.333333
9,9,10.0,"[[6.237871766090393, 5.0], [6.795262098312378,...","[5.0, 7.0, 9.0, 11.0, 15.0]",0,0,0.000000
